In [6]:
 %config cellreset on

------------- Classes -----------------
------------- Defs -----------------


# TableGen Language

### References
* [Programming](https://llvm.org/docs/TableGen/ProgRef.html)
* [Writing Tablegen backend](https://llvm.org/docs/TableGen/BackGuide.html)
* [Internals of Tablgen]()

### What tablegen solves?
### How tablegen solves it?

### Tablegen Lang
#### Overview
    * Domain Specific Lang for any work that has static structural data
    * Models custom data layouts, no actions describes(mostly)
    * Analogus to SQL
#### A. lexical 
   ##### 1. Writing Literals, identifiers, Special character sequence 
   ##### 2. Writing Bang Operators
       * expressions processing
   ##### 3. Composition of Files 
       * Support of Simple Preprocessor
#### B. Types
* Static Types -> same variable in single scope wont have to different types!
* Every "Value" defined will have type!
    * Simple Data types
        * bit
        * int
    * Composite Types
        * bits<>
        * string
        * list
        * dag
            + Dependency graph
            + Directed Acyclic Graph
            + Each Node is Operator
            + Each Edge is Argument which can be other dag or literal or identifer (type is must)
            
### Expressions
#### Left Side of Expression
* Name_Of_Record (optional) -> Record  = Record Template 
    + Anonymous Record
    + Named Record
    + Iterated Named Record (with Paste operator #)
* Subvalue from bits<>,list using {} , \[\] and <value>.field
    

In [55]:
class Foo{
    int x = 0;
    int y = 0;
}
def FooRec : Foo;   //  Record Def (LHS) : Record Template (RHS)

class xfoo : Foo{
    let x = 1;
}
class yfoo : Foo{
    let y = 1;
}
class Bar {}

def     : Bar,yfoo;      // Anonymous Record
def xyf : Bar,xfoo;      // Named Record
foreach i = 0...3 in
    def xy#i : Bar,xfoo; // Paste Operator
    
// def xyf : Bar,xfoo,yfoo;

------------- Classes -----------------
class Bar {
}
class Foo {
  int x = 0;
  int y = 0;
}
class xfoo {	// Foo
  int x = 1;
  int y = 0;
}
class yfoo {	// Foo
  int x = 0;
  int y = 1;
}
------------- Defs -----------------
def FooRec {	// Foo
  int x = 0;
  int y = 0;
}
def anonymous_0 {	// Bar Foo yfoo
  int x = 0;
  int y = 1;
}
def xy0 {	// Bar Foo xfoo
  int x = 1;
  int y = 0;
}
def xy1 {	// Bar Foo xfoo
  int x = 1;
  int y = 0;
}
def xy2 {	// Bar Foo xfoo
  int x = 1;
  int y = 0;
}
def xy3 {	// Bar Foo xfoo
  int x = 1;
  int y = 0;
}
def xyf {	// Bar Foo xfoo
  int x = 1;
  int y = 0;
}


In [36]:
// Record Creation using custom Layout
class Register{
    int id = 0;
}
def Reg0 :  Register;

------------- Classes -----------------
class Register {
  int id = 0;
}
------------- Defs -----------------
def Reg0 {	// Register
  int id = 0;
}


In [56]:
// Inheritance 
// Bang Operators
class GPRegister <int regId> {
    int id = regId;
}
class Register <int regId> :GPRegister <regId> {
    int xid = !mul(regId,5);
    int yid = !add(!mul(regId,8),5);    
}
def Reg0 :  Register<1>;
def Reg1 :  Register<2>;

------------- Classes -----------------
class GPRegister<int GPRegister:regId = ?> {
  int id = GPRegister:regId;
}
class Register<int Register:regId = ?> {	// GPRegister
  int id = Register:regId;
  int xid = !mul(Register:regId, 5);
  int yid = !add(!mul(Register:regId, 8), 5);
}
------------- Defs -----------------
def Reg0 {	// GPRegister Register
  int id = 1;
  int xid = 5;
  int yid = 13;
}
def Reg1 {	// GPRegister Register
  int id = 2;
  int xid = 10;
  int yid = 21;
}


In [57]:
// For Loop, Iteration-Induction
class GPRegister <int regId> {
    int id = regId;
}

class Register <int regId> :GPRegister <regId> {
    int xid = !mul(regId,5);
}

// RegisterFile
foreach i = 0-5 in
def Register#i : Register<i>;

------------- Classes -----------------
class GPRegister<int GPRegister:regId = ?> {
  int id = GPRegister:regId;
}
class Register<int Register:regId = ?> {	// GPRegister
  int id = Register:regId;
  int xid = !mul(Register:regId, 5);
}
------------- Defs -----------------
def Register0 {	// GPRegister Register
  int id = 0;
  int xid = 0;
}
def Register1 {	// GPRegister Register
  int id = 1;
  int xid = 5;
}
def Register2 {	// GPRegister Register
  int id = 2;
  int xid = 10;
}
def Register3 {	// GPRegister Register
  int id = 3;
  int xid = 15;
}
def Register4 {	// GPRegister Register
  int id = 4;
  int xid = 20;
}
def Register5 {	// GPRegister Register
  int id = 5;
  int xid = 25;
}


### Right Side Of Expressions
* This are Record templates for which we want to create instance
* e.g. class or list of comma seperated classes ( with restrictions*)
#### Statements
##### Define Record Template
    * class - Define record template
    * multiclass - Define multiple records 
    * When to use "class with template arg" vs multiclass ?
       - Analogy from C, class maps to struct where multiclass maps to macro
##### Define Record
    * def  - define concrete record
    * defm - Use multiclass to define multiple records, Name of record will be prepend
    * defset - collect a set of records
    * defvar - defines a global variable
##### Edit defined record/class
    * let - "override fields of class or record"
        + field to be overriden must be defined already
        + Late Binding in scope
        + Can not override template arguments!
#### Iterate statement ( not control flow)
    * foreach
#### Conditional Selection  (not control flow)
    * if
#### Helper
    * dump
    * assert

In [46]:
// Templating 
class Register <int regId> {
    int id = regId;
}
def Reg0 :  Register<0>;
def Reg1 :  Register<1>;

class Ops{}
def ADD : Ops;

class BinOpReg <dag d> {
  dag BinPat = d;
}
def ADDrr : BinOpReg<(ADD Reg0, Reg1)>;

------------- Classes -----------------
class BinOpReg<dag BinOpReg:d = ?> {
  dag BinPat = BinOpReg:d;
}
class Ops {
}
class Register<int Register:regId = ?> {
  int id = Register:regId;
}
------------- Defs -----------------
def ADD {	// Ops
}
def ADDrr {	// BinOpReg
  dag BinPat = (ADD Reg0, Reg1);
}
def Reg0 {	// Register
  int id = 0;
}
def Reg1 {	// Register
  int id = 1;
}


In [54]:
class Instruction<bits<4> OpCode> {
    bits<16> Encoding;
    let Encoding{15-12} = OpCode;
    bit isAtomic = 0;
}

multiclass BinaryInstruction<bits<4> OpCode> {
    def rr : Instruction<OpCode>;
    def rm : Instruction<OpCode>;      
}

multiclass AtomicBinInstruction<bits<4> OpCode>{
    let isAtomic = 1 in {
        def Xrr : Instruction<OpCode>;
        def Xrm : Instruction<OpCode>;
    }
}
// Notice difference in expression ADD instantiates for each record
defm ADD : BinaryInstruction<0x1>,AtomicBinInstruction<0x1>;
// defm ADD : BinaryInstruction<0x2>;  // Error

------------- Classes -----------------
class Instruction<bits<4> Instruction:OpCode = { ?, ?, ?, ? }> {
  bits<16> Encoding = { Instruction:OpCode{3}, Instruction:OpCode{2}, Instruction:OpCode{1}, Instruction:OpCode{0}, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? };
  bit isAtomic = 0;
}
------------- Defs -----------------
def ADDXrm {	// Instruction
  bits<16> Encoding = { 0, 0, 0, 1, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? };
  bit isAtomic = 1;
}
def ADDXrr {	// Instruction
  bits<16> Encoding = { 0, 0, 0, 1, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? };
  bit isAtomic = 1;
}
def ADDrm {	// Instruction
  bits<16> Encoding = { 0, 0, 0, 1, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? };
  bit isAtomic = 0;
}
def ADDrr {	// Instruction
  bits<16> Encoding = { 0, 0, 0, 1, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? };
  bit isAtomic = 0;
}


In [58]:
   // Let
   // Late Binding
   class Foo {
      string salutation = "Hi";
      string message = salutation#", world!";
    }

    def TimedBinding : Foo {
      let salutation = "Hello";
    }
    
    let salutation = "Hello but late" in 
        def  LateBindingFoo : Foo;

//    Expect Error : Value 'nosalute' unknown!
//    let nosalute = "Bon!" in
//        def errFoo: Foo;

------------- Classes -----------------
class Foo {
  string salutation = "Hi";
  string message = !strconcat(salutation, ", world!");
}
------------- Defs -----------------
def LateBindingFoo {	// Foo
  string salutation = "Hello but late";
  string message = "Hello but late, world!";
}
def TimedBinding {	// Foo
  string salutation = "Hello";
  string message = "Hello, world!";
}


In [ ]:
// RECURSION DOES NOT WORK!
class Base <int K>{
    int ret = !if(!cond(!eq(K,0):1,!eq(K,1):1),1,Base<!sub(K,1)>.ret);
}
class IndK <int K>{
    int BaseValue = !add(Base<K>.ret,K);
}
//def :IndK<3>;  // This is crash 

In [59]:
// Late Eval with iteration
class Example<list<int> nums> {
  list<int> doubled =
      !foreach(x, nums, !add(x, x));
}

def MyNums : Example<[4, 1, 9, -3]>;

------------- Classes -----------------
class Example<list<int> Example:nums = ?> {
  list<int> doubled = !foreach(x, Example:nums, !add(x, x));
}
------------- Defs -----------------
def MyNums {	// Example
  list<int> doubled = [8, 2, 18, -6];
}


In [60]:
// Enumeration with Left Folding
class Enumfy <list<string> startList, list<string> inList,string Namespace> {
    list<string> ret = !foldl(startList, inList, outList, element,!listconcat(outList, [Namespace#"::__" #element#"__"]));
}

def Opcodes : Enumfy<[],["ADD","SUB","MUL","DIV"],"OPCODE">;
def Modes   : Enumfy<[],["CPU","DPU","GPU","FPGA"],"MODE">;

------------- Classes -----------------
class Enumfy<list<string> Enumfy:startList = ?, list<string> Enumfy:inList = ?, string Enumfy:Namespace = ?> {
  list<string> ret = !foldl(Enumfy:startList, Enumfy:inList, outList, element, !listconcat(outList, [!strconcat(Enumfy:Namespace, !strconcat("::__", !strconcat(element, "__")))]));
}
------------- Defs -----------------
def Modes {	// Enumfy
  list<string> ret = ["MODE::__CPU__", "MODE::__DPU__", "MODE::__GPU__", "MODE::__FPGA__"];
}
def Opcodes {	// Enumfy
  list<string> ret = ["OPCODE::__ADD__", "OPCODE::__SUB__", "OPCODE::__MUL__", "OPCODE::__DIV__"];
}


### How Are records built  (Taken from Prog.ref)
* Build the record name (NameValue) and create an empty record.
* Parse the parent classes in the ParentClassList from left to right, visiting each parent class’s ancestor classes from top to bottom.
* Add the fields from the parent class to the record.
* Substitute the template arguments into those fields.
* Add the parent class to the record’s list of inherited classes.
* Apply any top-level let bindings to the record. Recall that top-level bindings only apply to inherited fields.
* Parse the body of the record.
* Add any fields to the record.
* Modify the values of fields according to local let statements.
* Define any defvar variables.
* Make a pass over all the fields to resolve any inter-field references.
* Add the record to the final record list.


### What Tablegen-Backend Generates for LLVM Backend/Clang?
* 